In [1]:
!make

Building application "ttn-sensors" for "b-l072z-lrwan1" with MCU "stm32".

[INFO] cloning stm32cmsis
Cloning into '/home/jovyan/work/training/iot-lab-training/riot/RIOT/build/stm32/cmsis/l0'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 245 (delta 196), reused 230 (delta 181), pack-reused 0
Receiving objects: 100% (245/245), 267.72 KiB | 3.43 MiB/s, done.
Resolving deltas: 100% (196/196), done.
HEAD is now at 532d969 Release v1.9.2
[INFO] updating stm32cmsis /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/stm32/cmsis/l0/.pkg-state.git-downloaded
echo 532d96973e7b2ae5546a2c88cb216429f74f5f5d   > /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/stm32/cmsis/l0/.pkg-state.git-downloaded
[INFO] patch stm32cmsis
[INFO] cloning semtech-loramac
Cloning into '/home/jovyan/work/training/iot-lab-training/riot/RIOT/build/pkg/semtech-loramac'...
remote: Enumerating obje

### Start an experiment on IoT-LAB

Now that we have a ready application, we can try it on the real hardware provided remotely by IoT-LAB.

1. Submit an experiment with one LoRa device on IoT-LAB:

In [2]:
!iotlab-experiment submit -n "Saclay" -d 10 -l 2,site=saclay+archi=st-lrwan1:sx1276

Using custom api_url: https://www.iot-lab.info/api/
{
    "id": 304272
}


2. Wait for the experiment to be in the "Running" state:

In [3]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout

Using custom api_url: https://www.iot-lab.info/api/
Waiting that experiment 304272 gets in state Running
"Running"


**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later.

3. Open a new terminal with the menu `File > New > Terminal` and in the terminal, run the following command to connect to the serial link of the LoRa board:

**It is normal if nothing is printed: there's normally no firmware running on the board. The firmware will be flashed during the next step**

**Keep this command running until the end of this notebook.**

4. Build and flash the application on the LoRa board:

In [13]:
!make IOTLAB_NODE=auto flash-only

Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
iotlab-node --jmespath='keys(@)[0]' --format='lambda ret: exit(int(ret))'  --list saclay,st-lrwan1,10 --flash /home/jovyan/work/training/iot-lab-training/riot/lorawan/ttn-sensors/bin/b-l072z-lrwan1/ttn-sensors.bin
Using custom api_url: https://www.iot-lab.info/api/


Once the flashing is complete, in the terminal, you should see the messages corresponding to the join procedure ("Join procedure succeeed") and the messages sent every 20s.

If the device cannot join, verify the configured EUIs and the application key.

If everything works as expected, you should see the decoded "This is RIOT!" messages appear in the **Live Data** tab of your application:


<figure style="text-align:center">
    <img src="images/ttn-decoded-messages-received.png" width="1262" align="center"/><br/><br/>
    <figcaption><em>Decoded messages received in the TTN console</em></figcaption>
</figure>

### Read the sensor values

Let's add the final step: read some sensor values from the [X-Nucleo extension shield](https://www.st.com/en/ecosystems/x-nucleo-iks01a2.html).

The X-Nucleo extension shield provides several ST Microelectronics sensors including the **HTS221** which measures temperature and humidity. The microcontroller interacts with this sensor using the [I2C serial bus](https://en.wikipedia.org/wiki/I%C2%B2C).

Let's now edit the [Makefile](Makefile) and [main.c](main.c) to read this sensor.

1. To be able to use the _hts221_ sensor with RIOT, you must first load the corresponding RIOT module in the application's [Makefile](Makefile). Add the following line to the Makefile (under the addition of the `ztimer_msec` module):


```mk
USEMODULE += hts221
```

All dependency modules required by the sensor driver will be loaded automatically during compilation, in particular the I2C bus driver used by the board to communicate with the sensor.

2. In the [main.c](main.c) file, Add the necessary header inclusions first (just below `#includes "semtech_loramac.h"`):


```c
#include "hts221.h"
#include "hts221_params.h"
```


3. Declare the variable containing the driver descriptor of the sensor (below the declaration of the `loramac` variable):

```c
static hts221_t hts221;
```

4. At the very beginning of the `main` function, add this sensor driver initialization sequence:

```c
    if (hts221_init(&hts221, &hts221_params[0]) != HTS221_OK) {
        puts("Sensor initialization failed");
        return 1;
    }

    if (hts221_power_on(&hts221) != HTS221_OK) {
        puts("Sensor initialization power on failed");
        return 1;
    }

    if (hts221_set_rate(&hts221, hts221.p.rate) != HTS221_OK) {
        puts("Sensor continuous mode setup failed");
        return 1;
    }
```

This sequence is adapted from the [hts221 test appplication](https://github.com/RIOT-OS/RIOT/blob/master/tests/driver_hts221/main.c#L34-L45).

5. Finally, add the following code at the beginning of the `while` loop to read the sensor data. You can also delete the line `char *message = "This is RIOT!"`
which is now useless:

```c
        /* do some measurements */
        uint16_t humidity = 0;
        int16_t temperature = 0;
        if (hts221_read_humidity(&hts221, &humidity) != HTS221_OK) {
            puts("Cannot read humidity!");
        }
        if (hts221_read_temperature(&hts221, &temperature) != HTS221_OK) {
            puts("Cannot read temperature!");
        }

        char message[64];
        sprintf(message, "H: %d.%d%%, T:%d.%dC",
                (humidity / 10), (humidity % 10),
                (temperature / 10), (temperature % 10));
        printf("Sending message '%s'\n", message);
```

6. Build and flash the new application to the LoRa board of your experiment:

In [14]:
!pwd

/home/jovyan/work/training/iot-lab-training/riot/lorawan/ttn-sensors


In [15]:
!make IOTLAB_NODE=auto flash

Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
Building application "ttn-sensors" for "b-l072z-lrwan1" with MCU "stm32".

"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/pkg/semtech-loramac
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/pkg/semtech-loramac/src/boards/mcu -f /home/jovyan/work/training/iot-lab-training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_arch
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/pkg/semtech-loramac/src/system/crypto -f /home/jovyan/work/training/iot-lab-training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_crypto
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/pkg/semtech-loramac/src/mac -f /home/jovyan/work/training/iot-lab-training/riot/RIOT/pkg/semtech-loramac/Makefile.loramac_mac
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/build/pkg/se

Now you can switch back to the terminal that is connected to the serial port of the LoRa board: **a new join procedure should be performed and the sensor data are sent every 20s to TTN**.

<figure style="text-align:center">
    <img src="images/ttn-sensor-data.png" width="1332" align="center"/><br/><br/>
    <figcaption><em>Sensor data received in the TTN console</em></figcaption>
</figure>


### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [16]:
!iotlab-experiment stop

Using custom api_url: https://www.iot-lab.info/api/
{
    "id": 304134,
    "status": "Delete request registered"
}


The serial link connection through SSH will be closed automatically.